<a href="https://colab.research.google.com/github/sira4154/AI4BA/blob/main/Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

#Preprocessing

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


dataset : https://grouplens.org/datasets/movielens/latest/

In [3]:
movies_df = pd.read_csv("/content/drive/MyDrive/movie/movies.csv")
ratings_df = pd.read_csv("/content/drive/MyDrive/movie/ratings.csv")

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_df.shape

(58098, 3)

In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [7]:
ratings_df.shape

(27753444, 4)

So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy extract function that Pandas has.

In [8]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


Let's look at the result!

In [9]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


###Drop Genres column 

In [10]:
movies_df = movies_df.drop('genres', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


##Check years

In [11]:
s = movies_df["year"].value_counts() #sort_year
s.sort_index()

1874       1
1878       1
1883       1
1887       1
1888       4
        ... 
2014    2297
2015    2312
2016    2219
2017    1891
2018     861
Name: year, Length: 134, dtype: int64

Check duplicate and drop

In [12]:
duplicate = movies_df[movies_df.duplicated(["title"], keep = False)]
duplicate.sort_values(by = ["title"])

,movieId,title,year
29798,128878,1,2014
24351,113214,1,2013
15321,77846,12 Angry Men,1997
1178,1203,12 Angry Men,1957
35224,141816,12 Chairs,1976
...,...,...,...
11910,53154,Zoo,2007
41429,156553,Zoom,2015
11241,47384,Zoom,2006
5801,5899,Zulu,1964


In [13]:
movies_df = movies_df.drop_duplicates(subset = ["title"], keep = "last")

In [14]:
movies_df.shape

(54635, 3)

In [15]:
movies_df.duplicated(["title"]).sum()

0

Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

###Drop timestamp

In [16]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [17]:
ratings_df.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [18]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


##Collaborative Filtering

Now, time to start our work on recommendation systems. 

The first technique we're going to take a look at is called **Collaborative Filtering**, which is also known as **User-User Filtering**. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

-   Select a user with the movies the user has watched
-   Based on his rating to movies, find the top X neighbours 
-   Get the watched movie record of the user for each neighbour.
-   Calculate a similarity score using some formula
-   Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [89]:
userInput = [
            {'title':'Iron Man 3', 'rating':4},
            {'title':'Jumanji: Welcome to the Jungle', 'rating':4},
            {'title':'Spider-Man', 'rating':4.5},
            {'title':"Star Wars: The Last Jedi", 'rating':5},
            {'title':'John Wick', 'rating':4.5},
            {'title':'Captain Marvel', 'rating': 3},
            {'title':'Ant-Man', 'rating':4},
            {'title':'Thor: Ragnarok', 'rating':4.5},
            {'title':'Captain America: Civil War', 'rating':4},
            {'title':'Jurassic World', 'rating':4.5},
            {'title':'Forrest Gump', 'rating': 5},
            {'title':'Saving Private Ryan', 'rating':4.5},
            {'title':'The Conjuring 2', 'rating':2},
            {'title':'The Meg', 'rating':1.5},
            {'title':'Ant-Man and the Wasp', 'rating':4},
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Iron Man 3,4.0
1,Jumanji: Welcome to the Jungle,4.0
2,Spider-Man,4.5
3,Star Wars: The Last Jedi,5.0
4,John Wick,4.5
5,Captain Marvel,3.0
6,Ant-Man,4.0
7,Thor: Ragnarok,4.5
8,Captain America: Civil War,4.0
9,Jurassic World,4.5


##Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.



In [90]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,movieId,title,rating
0,356,Forrest Gump,5.0
1,2028,Saving Private Ryan,4.5
2,5349,Spider-Man,4.5
3,102125,Iron Man 3,4.0
4,115149,John Wick,4.5
5,117529,Jurassic World,4.5
6,122900,Ant-Man,4.0
7,122910,Captain Marvel,3.0
8,122916,Thor: Ragnarok,4.5
9,122920,Captain America: Civil War,4.0


##The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [91]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
40,3,2028,5.0
122,4,356,3.5
325,4,2028,5.0
604,4,5349,5.0
1122,14,356,4.0


In [92]:
userSubset.shape

(221543, 3)

We now group up the rows by user ID.

In [93]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup.head()

,userId,movieId,rating
40,3,2028,5.0
122,4,356,3.5
325,4,2028,5.0
604,4,5349,5.0
1122,14,356,4.0
...,...,...,...
27752831,283220,356,5.0
27752857,283221,2028,4.0
27752942,283222,5349,2.5
27753002,283224,356,4.0


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [94]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [95]:
userSubsetGroup[100]

(4302,         userId  movieId  rating
 421194    4302      356     4.0
 421254    4302     2028     4.5
 421308    4302     5349     4.0
 421643    4302   102125     2.5
 421683    4302   115149     5.0
 421687    4302   117529     4.0
 421694    4302   122900     3.5
 421699    4302   122916     4.0
 421700    4302   122920     4.5
 421771    4302   179401     3.5
 421772    4302   179819     3.5)

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [96]:
userSubsetGroup = userSubsetGroup[0:10000]

In [97]:
userSubsetGroup[0:2]

[(72194,          userId  movieId  rating
  7035456   72194      356     4.5
  7035832   72194     2028     3.5
  7036477   72194     5349     3.5
  7037804   72194   102125     3.0
  7037949   72194   115149     3.5
  7037966   72194   117529     3.0
  7037989   72194   122900     3.0
  7037994   72194   122916     3.5
  7037996   72194   122920     3.5
  7038117   72194   159858     3.0
  7038229   72194   179401     2.5
  7038230   72194   179819     3.0
  7038265   72194   188301     3.0
  7038274   72194   192307     2.0), (186859,           userId  movieId  rating
  18296557  186859      356     5.0
  18297038  186859     2028     4.0
  18297844  186859     5349     4.5
  18299820  186859   102125     3.0
  18300028  186859   115149     4.5
  18300045  186859   117529     0.5
  18300079  186859   122900     2.5
  18300084  186859   122916     4.0
  18300086  186859   122920     3.0
  18300316  186859   159858     2.0
  18300491  186859   179401     3.5
  18300496  186859   179819

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient

In [98]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient คำนวณ
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    #ให้ x คือ 
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [99]:
temp_df

,movieId,title,rating
1,2028,Saving Private Ryan,4.5
2,5349,Spider-Man,4.5
9,122920,Captain America: Civil War,4.0


###calculation example

In [100]:
len(temp_df)

3

In [101]:
tempRatingList

[4.5, 4.5, 4.0]

In [102]:
tempGroupList

[2.0, 2.5, 2.0]

In [103]:
Sxx

0.1666666666666643

In [104]:
Syy

0.16666666666666607

In [105]:
Sxy

0.08333333333333215

In [106]:
nRatings

3

In [107]:
axx = ((1.5**2) + (4.5 ** 2))-((1.5+4.5)**2)/2
print(axx)

4.5


In [108]:
ayy = ((2**2) + (4 ** 2))-((2+4)**2)/2
print(ayy)

2.0


In [109]:
axy = ((1.5*2) + (4.5*4)) - ((1.5+4.5) * (2+4))/2
print(axy)

3.0


In [110]:
p_corr = axy / sqrt(axx*ayy)
print(p_corr)

1.0


###result similarity 

In [111]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF)) # reset index
pearsonDF.head()

,similarityIndex,userId
0,0.655977,72194
1,0.338640,186859
2,0.411113,220709
3,0.395741,13534
4,0.663166,19924


In [112]:
pearsonDF.tail()

,similarityIndex,userId
9995,-0.654654,39010
9996,-0.866025,39017
9997,0.277350,39060
9998,0.500000,39087
9999,0.500000,39124


##The top x similar users to input user
Now let's get the top 50 users that are most similar to the input.

In [113]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
8113,1.0,3706
8408,1.0,9082
9093,1.0,22231
4747,1.0,212650
8445,1.0,9863


In [114]:
topUsers.tail()

,similarityIndex,userId
8683,1.0,14234
8664,1.0,13778
8635,1.0,13323
8620,1.0,13006
8480,1.0,10507


##Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [115]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head(659)

,similarityIndex,userId,movieId,rating
0,1.0,3706,47,4.0
1,1.0,3706,50,3.5
2,1.0,3706,70,3.5
3,1.0,3706,110,5.0
4,1.0,3706,208,2.0
...,...,...,...,...
654,1.0,212650,176751,4.5
655,1.0,212650,177497,4.0
656,1.0,212650,177593,4.5
657,1.0,212650,177763,3.0


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [116]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,3706,47,4.0,4.0
1,1.0,3706,50,3.5,3.5
2,1.0,3706,70,3.5,3.5
3,1.0,3706,110,5.0,5.0
4,1.0,3706,208,2.0,2.0


In [117]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,30.0,123.0
2,18.0,59.0
3,5.0,14.5
5,6.0,16.5
6,11.0,43.0


In [118]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,4.100000,1
2,3.277778,2
3,2.900000,3
5,2.750000,5
6,3.909091,6


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [119]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
3415,5.0,3415
111757,5.0,111757
3546,5.0,3546
6669,5.0,6669
26150,5.0,26150
6643,5.0,6643
112473,5.0,112473
26258,5.0,26258
32456,5.0,32456


In [120]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
3328,3415,"Mirror, The (Zerkalo)",1975
3456,3546,What Ever Happened to Baby Jane?,1962
6534,6643,Tokyo Story (Tôkyô monogatari),1953
6560,6669,Ikiru,1952
8646,26150,Andrei Rublev (Andrey Rublyov),1969
8729,26258,"Topo, El",1970
9030,26743,Only Yesterday (Omohide poro poro),1991
9925,32456,"Pom Poko (a.k.a. Raccoon War, The) (Heisei tan...",1994
23915,111757,I Like Killing Flies,2004
24115,112473,Stuart: A Life Backward,2007


In [121]:
movie_df2 = recommendation_df.rename(columns={recommendation_df.columns[1]: 'id'})

In [122]:
movie_recom =movie_df2.merge(movies_df, left_on='id', right_on="movieId", how='inner')
movie_recom.head(20)

,weighted average recommendation score,id,movieId,title,year
0,5.0,3415,3415,"Mirror, The (Zerkalo)",1975
1,5.0,111757,111757,I Like Killing Flies,2004
2,5.0,3546,3546,What Ever Happened to Baby Jane?,1962
3,5.0,6669,6669,Ikiru,1952
4,5.0,26150,26150,Andrei Rublev (Andrey Rublyov),1969
5,5.0,6643,6643,Tokyo Story (Tôkyô monogatari),1953
6,5.0,112473,112473,Stuart: A Life Backward,2007
7,5.0,26258,26258,"Topo, El",1970
8,5.0,32456,32456,"Pom Poko (a.k.a. Raccoon War, The) (Heisei tan...",1994
9,5.0,26743,26743,Only Yesterday (Omohide poro poro),1991


In [123]:
movie_recom["year"] = pd.to_numeric(movie_recom['year'])
top10 = movie_recom[movie_recom["year"] >= 2015]
top10.head(10)

,weighted average recommendation score,id,movieId,title,year
22,5.0,122896,122896,Pirates of the Caribbean: Dead Men Tell No Tales,2017.0
23,5.0,122912,122912,Avengers: Infinity War - Part I,2018.0
64,5.0,150548,150548,Sherlock: The Abominable Bride,2016.0
71,5.0,156371,156371,Everybody Wants Some,2016.0
83,5.0,166461,166461,Moana,2016.0
86,5.0,163134,163134,Your Name.,2016.0
87,5.0,151455,151455,Eddie the Eagle,2016.0
89,5.0,183897,183897,Isle of Dogs,2018.0
171,4.5,156387,156387,Sing Street,2016.0
176,4.5,143713,143713,My Mother,2015.0


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages

-   Takes other user's ratings into consideration
-   Doesn't need to study or extract information from the recommended item
-   Adapts to the user's interests which might change over time

##### Disadvantages

-   Approximation function can be slow
-   There might be a low of amount of users to approximate
-   Privacy issues when trying to learn the user's preferences
